# Diodenkennlinie Messung mit DAC und MCC118

## Einführung

### Was ist eine Diodenkennlinie?
Die **Diodenkennlinie** beschreibt den Zusammenhang zwischen Spannung und Strom einer Diode. Sie zeigt das charakteristische **exponentiell ansteigende Verhalten** im Durchlassbereich und den **Sperrbereich** bei negativen Spannungen.

### Ziel des Experiments
Wir messen die I-V-Charakteristik einer Diode, indem wir:
1. Mit einem **DAC** verschiedene Spannungen erzeugen
2. Mit dem **MCC118** die Diodenspannung messen  
3. Den Diodenstrom über das Ohmsche Gesetz berechnen
4. Die Kennlinie grafisch darstellen

### Physikalischer Hintergrund
Die Diodenkennlinie folgt der **Shockley-Gleichung**:

**I = I_s × (e^(qV/nkT) - 1)**

Dabei ist:
- **I_s**: Sättigungsstrom (sehr klein)
- **q**: Elementarladung  
- **V**: Diodenspannung
- **n**: Idealitätsfaktor (1-2)
- **k**: Boltzmann-Konstante
- **T**: absolute Temperatur

---

## Schritt 1: Bibliotheken importieren

Wir benötigen verschiedene Python-Bibliotheken für die Hardware-Steuerung und Datenanalyse:

In [1]:
from __future__ import print_function  # Python 2/3 Kompatibilität
import spidev                          # SPI-Kommunikation mit DAC
import time                           # Wartezeiten
import RPi.GPIO as GPIO               # GPIO-Steuerung für Chip Select
import matplotlib.pyplot as plt       # Diagramme erstellen

from daqhats import mcc118, OptionFlags, HatIDs    # MCC118 Hardware-Bibliothek
from daqhats_utils import select_hat_device        # Device-Auswahl

## Schritt 2: DAC-Parameter definieren

### Was ist ein DAC?
Ein **DAC (Digital-Analog-Converter)** wandelt digitale Werte in analoge Spannungen um. Unser DAC hat eine **12-Bit Auflösung** (4096 Stufen) und kann Spannungen bis **10,7V** erzeugen.

### Hardware-Parameter:


In [2]:
# === DAC Hardware-Parameter ===
CS_PIN = 22              # GPIO Pin für Chip Select (SPI)
MAX_DAC_VALUE = 4095     # Maximum DAC-Wert (12-Bit: 2^12 - 1)
MAX_SPANNUNG = 10.7      # Maximale Ausgangsspannung in Volt

# Auflösung berechnen
SPANNUNG_PRO_BIT = MAX_SPANNUNG / MAX_DAC_VALUE  # ≈ 2.61 mV pro Bit

### Berechnung der Ausgangsspannung:
**U_out = (DAC_Wert / 4095) × 10.7V**

Beispiele:
- DAC-Wert 0 → 0V
- DAC-Wert 2048 → 5.35V  
- DAC-Wert 4095 → 10.7V

---

## Schritt 3: SPI-Kommunikation einrichten

### SPI-Konfiguration
Der DAC kommuniziert über die **SPI-Schnittstelle** mit speziellen Timing-Anforderungen:

In [3]:
# === SPI-Schnittstelle konfigurieren ===
spi = spidev.SpiDev()
spi.open(0, 0)              # Bus 0, Device 0
spi.max_speed_hz = 1000000  # 1 MHz Übertragungsgeschwindigkeit
spi.mode = 0b00             # SPI Mode 0 (CPOL=0, CPHA=0)

# === GPIO für Chip Select ===
GPIO.setmode(GPIO.BCM)      # BCM Pin-Nummerierung
GPIO.setup(CS_PIN, GPIO.OUT)
GPIO.output(CS_PIN, GPIO.HIGH)  # CS initial auf HIGH (inaktiv)

### SPI-Modi Erklärung:
- **Mode 0**: CPOL=0, CPHA=0 (Clock LOW, Daten bei steigender Flanke)
- **CPOL**: Clock-Polarität (0=LOW, 1=HIGH im Ruhezustand)
- **CPHA**: Clock-Phase (0=erste Flanke, 1=zweite Flanke)

---

## Schritt 4: DAC-Steuerfunktion

### DAC-Datenformat
Der DAC erwartet ein **16-Bit Datenpaket** mit folgender Struktur:

| Bit | 15 | 14 | 13 | 12 | 11-0 |
|-----|----|----|----|----|------|
| **Funktion** | Channel | Buffer | Gain | Shutdown | Data |

In [4]:
def write_dac(value):
    """
    Schreibt einen Wert an den DAC (0-4095)
    
    Parameter:
    value: DAC-Wert (0 bis 4095)
    
    DAC-Register-Aufbau (16 Bit):
    - Bit 15: Channel Select (0=A, 1=B) 
    - Bit 14: Buffer Control (1=buffered)
    - Bit 13: Gain Select (0=2x, 1=1x)
    - Bit 12: Shutdown (1=active, 0=shutdown)
    - Bit 11-0: Data (12-Bit DAC-Wert)
    """
    # Eingabe validieren
    assert 0 <= value <= MAX_DAC_VALUE, f"DAC-Wert muss zwischen 0 und {MAX_DAC_VALUE} liegen!"
    
    # === Kontrollbits zusammensetzen ===
    control = 0
    control |= 0 << 15  # Channel A auswählen (0)
    control |= 1 << 14  # Buffered Mode aktivieren (1)
    control |= 0 << 13  # Gain = 2x für vollen Spannungsbereich (0)
    control |= 1 << 12  # Normal Operation, nicht Shutdown (1)
    
    # === Datenpaket erstellen ===
    data = control | (value & 0xFFF)  # 12-Bit Daten hinzufügen
    high_byte = (data >> 8) & 0xFF    # Obere 8 Bit
    low_byte = data & 0xFF            # Untere 8 Bit

    # Debug-Ausgabe (optional)
    spannung = (value / MAX_DAC_VALUE) * MAX_SPANNUNG
    print(f"DAC: {value:4d} → {spannung:.3f}V | SPI: 0x{high_byte:02X}{low_byte:02X}")
    
    # === SPI-Übertragung ===
    GPIO.output(CS_PIN, GPIO.LOW)     # Chip Select aktivieren
    spi.xfer2([high_byte, low_byte])  # 16-Bit Daten senden
    GPIO.output(CS_PIN, GPIO.HIGH)    # Chip Select deaktivieren

### Warum Buffered Mode?
- **Buffered**: Höhere Strombelastbarkeit, stabile Spannung
- **Unbuffered**: Direkter Ausgang, kann bei Last schwanken

---

## Schritt 5: Schaltungsanalyse

### Messschaltung verstehen
Unsere Schaltung besteht aus:

```
DAC → [R_serie] → [Diode] → GND
                     ↑
                  MCC118 (Spannungsmessung)
```

### Stromberechnung mit Ohmschen Gesetz:

**U_DAC = U_Diode + U_Widerstand**

**U_Widerstand = I × R_serie**

**Daher: I = (U_DAC - U_Diode) / R_serie**

### Warum brauchen wir den Serienwiderstand?
1. **Strombegrenzung**: Schutz der Diode vor zu hohem Strom
2. **Messbarkeit**: Ermöglicht indirekte Strommessung
3. **Stabilität**: Verhindert Schwingungen im steilen Kennlinienbereich

---

## Schritt 6: Hauptmessung durchführen

### Messablauf programmieren


In [6]:
def main():
    """
    Hauptfunktion für die Diodenkennlinienmessung
    
    Ablauf:
    1. Messparameter vom Benutzer abfragen
    2. Hardware initialisieren
    3. Spannungsrampe fahren und dabei messen
    4. Strom aus Spannung berechnen  
    5. Kennlinien-Diagramme erstellen
    """
    print("=" * 50)
    print("    DIODENKENNLINIE MESSUNG")
    print("=" * 50)

    try:
        # === Messparameter eingeben ===
        print("\n📋 Messparameter eingeben:")
        r_serie = float(input("Serienwiderstand [Ω] (z.B. 100): "))
        anzahl_punkte = int(input("Anzahl Messpunkte (mind. 15): "))
        
        # Mindestanzahl prüfen
        if anzahl_punkte < 15:
            print("⚠️  Mindestens 15 Punkte für aussagekräftige Kennlinie!")
            anzahl_punkte = 15

        spannung_max = float(input(f"Maximale Spannung [V] (max {MAX_SPANNUNG:.1f}V): "))
        if spannung_max > MAX_SPANNUNG:
            print(f"⚠️  Begrenze auf {MAX_SPANNUNG:.1f}V.")
            spannung_max = MAX_SPANNUNG

        # === Hardware verbinden ===
        print("\n🔧 Verbinde Hardware...")
        address = select_hat_device(HatIDs.MCC_118)
        hat = mcc118(address)
        print(f"✅ MCC118 an Adresse {address} verbunden.")

        # === Messreihen vorbereiten ===
        eingestellte_spannungen = []  # U_DAC (Sollwert)
        diodenspannungen = []         # U_Diode (gemessen)
        stroeme = []                  # I_Diode (berechnet)

        print(f"\n📊 Starte Messung: 0V → {spannung_max:.1f}V in {anzahl_punkte} Schritten")
        print("=" * 80)
        print("  Nr. |  U_DAC [V] | U_Diode [V] | I_Diode [mA] | DAC-Wert")
        print("-" * 80)
        
        # === Messschleife ===
        for i in range(anzahl_punkte):
            # Spannung linear verteilen (0 bis spannung_max)
            spannung_dac = i * spannung_max / (anzahl_punkte - 1)
            
            # DAC-Wert berechnen
            dac_value = int((spannung_dac / MAX_SPANNUNG) * MAX_DAC_VALUE)
            
            # === DAC setzen ===
            write_dac(dac_value)
            time.sleep(0.05)  # Einschwingzeit abwarten (wichtig!)

            # === Diodenspannung messen ===
            spannung_diode = hat.a_in_read(7)  # MCC118 Kanal 7

            # === Strom berechnen ===
            strom = (spannung_dac - spannung_diode) / r_serie
            strom_ma = strom * 1000  # Umrechnung in mA für bessere Lesbarkeit

            # === Daten speichern ===
            eingestellte_spannungen.append(spannung_dac)
            diodenspannungen.append(spannung_diode)
            stroeme.append(strom)

            # === Fortschritt anzeigen ===
            print(f" {i+1:3d}. | {spannung_dac:8.3f} | {spannung_diode:9.5f} | "
                  f"{strom_ma:10.3f} | {dac_value:8d}")

        # === DAC auf 0V setzen (Sicherheit) ===
        write_dac(0)
        print("\n✅ Messung abgeschlossen!")

        # === Datenanalyse ===
        print(f"\n📈 Analysiere {len(stroeme)} Datenpunkte...")
        
        # Schwellenspannung finden (Strom > 1mA)
        schwellenspannung = None
        for u, i in zip(diodenspannungen, stroeme):
            if i > 0.001:  # 1mA Grenze
                schwellenspannung = u
                break
        
        if schwellenspannung:
            print(f"🔍 Schwellenspannung: ≈ {schwellenspannung:.3f}V")

        # === Diagramme erstellen ===
        print("📊 Erstelle Diagramme...")
        
        plt.figure(figsize=(14, 6))

        # === Subplot 1: Klassische I-V Kennlinie ===
        plt.subplot(1, 2, 1)
        plt.plot(diodenspannungen, [i*1000 for i in stroeme], 'ro-', 
                linewidth=2, markersize=4, label='Messdaten')
        plt.xlabel("Diodenspannung U_D [V]", fontsize=11)
        plt.ylabel("Diodenstrom I_D [mA]", fontsize=11) 
        plt.title("Diodenkennlinie (I-V)", fontsize=12, fontweight='bold')
        plt.grid(True, alpha=0.3)
        plt.legend()
        
        # Schwellenspannung markieren
        if schwellenspannung:
            plt.axvline(schwellenspannung, color='orange', linestyle='--', 
                       label=f'Schwelle ≈ {schwellenspannung:.3f}V')
            plt.legend()

        # === Subplot 2: Eingangs- vs. Ausgangskennlinie ===
        plt.subplot(1, 2, 2)
        plt.plot(eingestellte_spannungen, [i*1000 for i in stroeme], 'bo-',
                linewidth=2, markersize=4, label='I_D(U_DAC)')
        plt.xlabel("Eingangsspannung U_DAC [V]", fontsize=11)
        plt.ylabel("Diodenstrom I_D [mA]", fontsize=11)
        plt.title("Eingangsspannung vs. Strom", fontsize=12, fontweight='bold')
        plt.xlim(0, spannung_max)
        plt.grid(True, alpha=0.3)
        plt.legend()

        plt.tight_layout()
        plt.show()

        # === Zusammenfassung ausgeben ===
        max_strom = max(stroeme) * 1000
        max_spannung = max(diodenspannungen)
        print(f"\n📋 Messergebnisse:")
        print(f"   • Maximaler Strom: {max_strom:.2f} mA")
        print(f"   • Maximale Diodenspannung: {max_spannung:.3f} V")
        print(f"   • Serienwiderstand: {r_serie:.0f} Ω")
        print(f"   • Schwellenspannung: {schwellenspannung:.3f} V" if schwellenspannung else "   • Schwellenspannung: nicht erreicht")

    except KeyboardInterrupt:
        print("\n⚠️  Messung durch Benutzer abgebrochen.")
        write_dac(0)  # Sicherheit: DAC auf 0V
    except Exception as e:
        print(f"❌ Fehler: {e}")
        write_dac(0)  # Sicherheit: DAC auf 0V
    finally:
        # === Cleanup (immer ausführen!) ===
        try:
            write_dac(0)    # DAC auf 0V setzen
            GPIO.cleanup()  # GPIO zurücksetzen
            print("🧹 Hardware-Cleanup abgeschlossen.")
        except:
            pass

# === Messung starten ===
if __name__ == "__main__":
    main()

    DIODENKENNLINIE MESSUNG

📋 Messparameter eingeben:

⚠️  Messung durch Benutzer abgebrochen.
DAC:    0 → 0.000V | SPI: 0x5000
DAC:    0 → 0.000V | SPI: 0x5000


RuntimeError: Please set pin numbering mode using GPIO.setmode(GPIO.BOARD) or GPIO.setmode(GPIO.BCM)

## Schritt 7: Physikalische Interpretation

### Die Diodenkennlinie verstehen

#### Charakteristische Bereiche:

1. **Sperrbereich (V < 0)**:
   - Diode ist gesperrt
   - Nur minimaler Sperrstrom fließt (nA bis μA)
   - Strom praktisch konstant

2. **Schwellenbereich (0 < V < V_th)**:
   - Diode beginnt zu leiten
   - Exponentieller Stromanstieg
   - Schwellenspannung V_th ≈ 0.7V (Silizium)

3. **Durchlassbereich (V > V_th)**:
   - Diode leitet gut
   - Starker Stromanstieg bei kleinen Spannungsänderungen
   - Praktisch konstante Spannung bei steigendem Strom

#### Materialabhängige Schwellenspannungen:
- **Silizium (Si)**: V_th ≈ 0.7V
- **Germanium (Ge)**: V_th ≈ 0.3V  
- **LED (verschiedene Farben)**: V_th ≈ 1.8V - 3.5V
- **Schottky-Diode**: V_th ≈ 0.3V - 0.5V

### Mathematische Modellierung

#### Shockley-Gleichung (vereinfacht):
Für V > 0.1V kann die Gleichung vereinfacht werden zu:

**I ≈ I_s × e^(V/V_T)**

Dabei ist:
- **V_T = kT/q**: Temperaturspannung (≈ 26mV bei Raumtemperatur)
- **I_s**: Sättigungsstrom (materialabhängig)

#### Logarithmische Darstellung:
**ln(I) = ln(I_s) + V/V_T**

Dies erklärt die **exponentiell ansteigende Kennlinie**.

---

## Schritt 8: Fehlerquellen und Verbesserungen

### Häufige Probleme:

#### 1. **Selbsterwärmung der Diode**
- **Problem**: Hoher Strom → Erwärmung → Kennlinie verschiebt sich
- **Lösung**: Kurze Messpulse, Wartezeiten einhalten

#### 2. **Leitungswiderstände**
- **Problem**: Spannungsabfall an Kabeln verfälscht Messung
- **Lösung**: Kurze, dicke Leitungen verwenden

#### 3. **ADC-Auflösung**
- **Problem**: Kleine Spannungen werden ungenau gemessen
- **Lösung**: Höhere ADC-Auflösung, Verstärker verwenden

#### 4. **Temperaturabhängigkeit**
- **Problem**: Kennlinie ändert sich mit Temperatur
- **Lösung**: Konstante Temperatur, Temperaturkompensation

In [5]:
# Beispiel für verbesserte Messung mit Mittelwertbildung
def improved_measurement(hat, channel, num_samples=10):
    """
    Verbesserte Spannungsmessung mit Mittelwertbildung
    """
    samples = []
    for _ in range(num_samples):
        samples.append(hat.a_in_read(channel))
        time.sleep(0.001)  # Kurze Pause zwischen Messungen
    
    return sum(samples) / len(samples)  # Mittelwert

## Schritt 9: Erweiterte Auswertung

### Kennlinienparameter extrahieren

In [6]:



def analyze_diode_curve(voltages, currents):
    """
    Erweiterte Analyse der Diodenkennlinie
    """
    import numpy as np
    
    # Schwellenspannung bestimmen (verschiedene Kriterien)
    thresholds = {
        "1mA": find_threshold(voltages, currents, 0.001),
        "10mA": find_threshold(voltages, currents, 0.010),
        "1% Max": find_threshold(voltages, currents, max(currents) * 0.01)
    }
    
    # Differentieller Widerstand im Arbeitspunkt
    r_diff = calculate_differential_resistance(voltages, currents)
    
    # Idealitätsfaktor schätzen (aus Steigung)
    ideality_factor = estimate_ideality_factor(voltages, currents)
    
    return {
        "thresholds": thresholds,
        "r_diff": r_diff,
        "ideality": ideality_factor
    }

def find_threshold(voltages, currents, threshold_current):
    """Findet Schwellenspannung für gegebenen Strom"""
    for v, i in zip(voltages, currents):
        if i >= threshold_current:
            return v
    return None

## Schritt 10: Praktische Tipps

### Experimenteller Aufbau:
1. **Kurze Verbindungen**: Minimiert Störungen und Widerstände
2. **Abschirmung**: Reduziert Rauschen von externen Quellen
3. **Temperaturkontrolle**: Konstante Umgebungstemperatur
4. **Kalibrierung**: Multimeter als Referenz verwenden

### Sicherheitshinweise:
- **Maximale Spannung**: Nicht über Sperrspannung der Diode gehen
- **Maximaler Strom**: Thermal Runaway vermeiden
- **ESD-Schutz**: Antistatische Vorsichtsmaßnahmen

### Typische Messwerte:
- **1N4007 (Si-Diode)**: V_th ≈ 0.7V, I_max = 1A
- **1N4148 (Schaltdiode)**: V_th ≈ 0.65V, I_max = 200mA  
- **LED (rot)**: V_th ≈ 1.8V, I_nom = 20mA

---

## Zusammenfassung


1. **Hardware-Steuerung**: DAC über SPI programmieren
2. **Messtechnik**: Indirekte Strommessung über Spannungsabfall
3. **Datenanalyse**: Kennlinien erstellen und interpretieren
4. **Physik**: Halbleiterverhalten verstehen

### Weiterführende Experimente:
- **Temperaturabhängigkeit**: Kennlinie bei verschiedenen Temperaturen
- **Verschiedene Dioden**: Vergleich unterschiedlicher Diodentypen
- **AC-Analyse**: Kleinsignalverhalten untersuchen
- **Rückwärtskennlinie**: Sperrverhalten charakterisieren

Die Diodenkennlinie ist ein fundamentales Experiment in der Elektronik und bildet die Grundlage für das Verständnis komplexerer Halbleiterbauelemente wie Transistoren und integrierte Schaltungen.
        